# Batch processing

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import cellpy
from cellpy import prms
from cellpy import prmreader
from cellpy.utils import batch

In [ ]:
######################################################################
##                                                                  ##
##                       development                                ##
##                                                                  ##
######################################################################

from pathlib import Path
from pprint import pprint

# Use these when working on my work PC:
test_data_path = r"C:\Scripting\MyFiles\development_cellpy\testdata"
out_data_path = r"C:\Scripting\Processing\Test\out"

# Use these when working on my MacBook:
test_data_path = "/Users/jepe/scripting/cellpy/testdata"
out_data_path = "/Users/jepe/cellpy_data"

test_data_path = Path(test_data_path)
out_data_path = Path(out_data_path)

print(" SETTING SOME PRMS ".center(80, "="))
prms.Paths["db_filename"] = "cellpy_db.xlsx"
prms.Paths["cellpydatadir"] = test_data_path / "hdf5"
prms.Paths["outdatadir"] = out_data_path
prms.Paths["rawdatadir"] = test_data_path / "data"
prms.Paths["db_path"] = test_data_path / "db"
prms.Paths["filelogdir"] = test_data_path / "log"
pprint(prms.Paths)

In [ ]:
# prmreader.info()

In [ ]:
project = "prebens_experiment"
name = "test"
batch_col = "b01"

print(" INITIALISATION OF BATCH ".center(80, "="))
b = batch.init(name, project, batch_col=batch_col)
print(b)

In [ ]:
# setting some prms
b.experiment.export_raw = True
b.experiment.export_cycles = True
b.experiment.export_ica = True

In [ ]:
b.create_info_df()

In [ ]:
b.create_folder_structure()

In [ ]:
b.load_and_save_raw()

In [ ]:
b.make_summaries()
print("---FINISHED---")

In [ ]:
b.plot_summaries()

In [ ]:
b.experiment.status()

In [ ]:
b.summaries.head()

## Developing the CyclingSummaryPlotter

In [ ]:
import time
import pandas

from cellpy.utils.batch_tools import (
    batch_experiments,
    batch_exporters,
    batch_journals,
    batch_plotters,
    dumpers,
    engines,
)
from cellpy import log


In [ ]:
log.setup_logging(default_level="DEBUG")

In [ ]:
plotter = batch_plotters.CyclingSummaryPlotter(b.experiment)

In [ ]:
import bokeh.plotting

In [ ]:
bokeh.plotting.output_notebook()

In [ ]:
plotter.do()

In [ ]:
b.summaries.columns.get_level_values(0)

In [ ]:
discharge_capacity = b.summaries.discharge_capacity
charge_capacity = b.summaries.charge_capacity
coulombic_efficiency = b.summaries.coulombic_efficiency
ir_charge = b.summaries.ir_charge

In [ ]:
import itertools
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, Range1d, HoverTool
from bokeh.layouts import column
from bokeh.models.annotations import Legend
output_notebook()

In [ ]:
def create_legend(info, c, option="clean", use_index=False):
    mass, loading, label = info.loc[c, ["masses", "loadings", "labels"]]
    
    if use_index or not label:
        label = c.split("_")
        label = "_".join(label[1:])
        
    if option=="clean":
        return label
    
    if option == "mass":
        label = f"{label} ({mass:.2f} mg)"
    elif option == "loading":
        label = f"{label} ({loading:.2f} mg/cm2)"
    elif option == "all":
        label = f"{label} ({mass:.2f} mg) ({loading:.2f} mg/cm2)"
        
    return label

In [ ]:
c = "20160805_test001_45_cc"
x = create_legend(b.info_df, c, option="all")
#print(c)
#print(x)

In [ ]:
#b.info_df

In [ ]:
def look_up_group(info, c):
    g, sg = info.loc[c, ["groups", "sub_groups"]]
    return int(g), int(sg)


In [ ]:
import bokeh.palettes

In [ ]:
#palettes.brewer['YlGnBu']


In [ ]:
def create_plot_option_dicts(info, marker_types=None, colors=None, line_dash=None, size=None):
    """Create two dictionaries with plot-options.
    
    The first iterates colors (based on group-number), the second iterates
    through marker types.
    
    Returns: group_styles (dict), sub_group_styles (dict)
    """
    
    if marker_types is None:
        marker_types = ["circle", "square", "triangle", "invertedtriangle", "diamond", "cross", "asterix"]
        
    if line_dash is None:
        line_dash = [0, 0]
        
    if size is None:
        size = 12
        
    groups = info.groups.unique()
    number_of_groups = len(groups)
    if colors is None:
        if number_of_groups < 4:
            # print("using 3")
            colors = bokeh.palettes.brewer['YlGnBu'][3]
        else:
            # print(f"using {min(9, number_of_groups)}")
            colors = bokeh.palettes.brewer['YlGnBu'][min(9, number_of_groups)]
    
    sub_groups = info.sub_groups.unique()
    
    marker_it = itertools.cycle(marker_types)
    colors_it = itertools.cycle(colors)

    group_styles = dict()
    sub_group_styles = dict()

    for j in groups:
        color = next(colors_it)
        marker_options = {
            "line_color": color,
            "fill_color": color,
        }

        line_options = {
            "line_color": color,
        }
        group_styles[j] = {
            "marker": marker_options,
            "line": line_options,
        }

    for j in sub_groups:
        marker_type = next(marker_it)
        marker_options = {
            "marker": marker_type,
            "size": size,
        }

        line_options = {
            "line_dash": line_dash,
        }
        sub_group_styles[j] = {
            "marker": marker_options,
            "line": line_options,
        }
    return group_styles, sub_group_styles

group_styles, sub_group_styles = create_plot_option_dicts(b.info_df)

pprint(group_styles)
pprint(sub_group_styles)

In [ ]:
def create_summary_plot(data, info, group_styles, sub_group_styles,
                        title="Capacity", x_axis_label="Cycle number", y_axis_label="Capacity (mAh/g)",
                        width=900, height=400,
                        legend_option="clean",
                        legend_location="bottom_right",
                        x_range=None,
                        y_range=None,
                        tools = ["hover",]
                        ):

    discharge_capacity = None
    if isinstance(data, (list, tuple)):
        charge_capacity = data[0]
        if len(data) == 2:
            discharge_capacity = data[1]
    else:
        charge_capacity = data
        
    charge_source = ColumnDataSource(charge_capacity)
    if discharge_capacity is not None:
        discharge_source = ColumnDataSource(discharge_capacity)

    p = figure(title=title, width=width, height=height,
               # tools = tools,
               x_range = x_range,
               y_range = y_range,
               x_axis_label=x_axis_label,
               y_axis_label=y_axis_label)

    cols = charge_capacity.columns.get_level_values(0)
    if legend_option is not None:
        legend_collection = []
    
    for c in cols:
        g, sg = look_up_group(info, c)

        if legend_option is not None:
            legend_items = []
            l = create_legend(info, c, option=legend_option)
            #legend_option_dict = {"legend": f"{l}"}

        group_props = group_styles[g]
        sub_group_props = sub_group_styles[sg]

        ch_m = p.scatter(
            source=charge_source, 
            x="Cycle_Index", y=c, 
            #**legend_option_dict, # Remark! cannot use the same legend name as column name (defaults to a lookup)
            **group_props["marker"], # color
            **sub_group_props["marker"], # marker
        )
        
        ch_l = p.line(
            source=charge_source, 
            x="Cycle_Index", y=c, 
            **group_props["line"],
            **sub_group_props["line"],
        )
        
        if legend_option is not None:
            legend_items.extend([ch_m, ch_l])
        
        if discharge_capacity is not None:
            # creating a local copy so that I can do local changes
            group_props_marker_charge = group_props["marker"].copy()
            group_props_marker_charge["fill_color"] = None
            dch_m = p.scatter(
                source=discharge_source, 
                x="Cycle_Index", y=c, 
                **group_props_marker_charge,
                **sub_group_props["marker"],
            )
            
            dch_l = p.line(
            source=discharge_source, 
            x="Cycle_Index", y=c, 
            **group_props["line"],
            **sub_group_props["line"],
            )
            
            if legend_option is not None:
                legend_items.extend([dch_m, dch_l])
                
        if legend_option is not None:
            legend_collection.append((l, legend_items))
            
    if discharge_capacity is not None:
        print("(filled:charge) (open:discharge)")
        
    if legend_option is not None:
        
        legend = Legend(
            items=legend_collection,
            location=(10, 0)
        )
        p.add_layout(legend)
        p.legend.location = legend_location
        p.legend.click_policy = "hide"
    return p


In [ ]:
def plot_cycle_life_summary(info, summaries, width=900, height=800, height_fractions=[0.2, 0.5, 0.3]):
    
    discharge_capacity = summaries.discharge_capacity
    charge_capacity = summaries.charge_capacity
    coulombic_efficiency = summaries.coulombic_efficiency
    ir_charge = summaries.ir_charge

    h_eff = int(height_fractions[0] * height)
    h_cap = int(height_fractions[1] * height)
    h_ir  = int(height_fractions[2] * height)
    
    group_styles, sub_group_styles = create_plot_option_dicts(info)

    p_eff = create_summary_plot(
        coulombic_efficiency, info, group_styles, sub_group_styles,
        legend_option=None,title=None,x_axis_label=None, y_axis_label="Coulombic efficiency (%)",
        width=width, height=h_eff,
    )

    p_cap = create_summary_plot(
        (charge_capacity, discharge_capacity), info, group_styles, sub_group_styles,
        title=None, x_axis_label=None, height=h_cap, width=width,
        x_range=p_eff.x_range, 
    )

    p_ir = create_summary_plot(
        ir_charge, info, group_styles, sub_group_styles,
        legend_option=None,title=None,x_axis_label="Cycle number", y_axis_label="IR Charge (Ohm)",
        width=width, height=h_ir,
        x_range=p_eff.x_range,
    )

    p_eff.y_range.start, p_eff.y_range.end = 20, 120
    p_eff.xaxis.visible = False
    p_cap.xaxis.visible = False

    hover = HoverTool(tooltips=[
        ("cycle", "@Cycle_Index"),
        ("value", "$y"),
    ])

    p_eff.add_tools(hover)
    p_cap.add_tools(hover)
    p_ir.add_tools(hover)

    show(column(p_eff, p_cap, p_ir))

In [ ]:
width = 900
height = 800
info = b.info_df
height_fractions = [0.2, 0.5, 0.3]
plot_cycle_life_summary(info, b.summaries, width, height, height_fractions)